In [ ]:
import pandas as pd
import os.path
import json
import codecs
from tqdm import tqdm
from wordsegment import load, segment
load()


In [ ]:
import markdown

md = markdown.Markdown(extensions=["markdown_cjk_spacing.cjk_spacing"],
    extension_configs={'markdown_cjk_spacing.cjk_spacing':
            {'segment_break': True}})


In [ ]:
# loading datasets
def load_lang_df(lang):
    metadata_file = f"mandarin_files.json"
    df = pd.read_json(metadata_file, lines=True, orient="records")
    df.name=f"{lang} dataset"
    return df

target_dataset = load_lang_df("Mandarin Data")

In [ ]:
# total number of records
print(f"There are {target_dataset.shape[0]} records in the {target_dataset.name}")


In [ ]:
count_exist=0
count_non_exist=0
mainJsonData=[]
letters = "abcdefghijklmnopqrstuwvxyz"

for audioFileName, ftext in tqdm(zip(target_dataset['audio_filepath'], target_dataset['text']), total = len(target_dataset['audio_filepath'])):

    #print(ftext)
    
    newtext = ftext.replace(" ","")
    newtext = newtext.replace(" ","")
    #print(newtext)
    ntext = md.convert(newtext)
    #print(ntext)
    ntext = ntext.replace('<p>', '').replace('</p>', '')
    
    #ntext = " " + ntext
    
    #print(ntext)
    
    check_text = ntext.split(" ")
    
    if len(check_text) > 1 or ntext[0] in letters:
        last_text=""
        for subwrod in check_text:
            flag=0
            for alett in subwrod:
                if alett.lower() in letters:
                    segmented_text = segment(subwrod.lower())
                    #print(segmented_text)
                    joined_segmented_text = ' '.join(segmented_text)
                    flag=1
                    break
                                        
            if flag==0:
                joined_segmented_text = subwrod

            last_text+= " " + joined_segmented_text 
            #print(audioFileName)
            #print(ftext)
            #print(last_text)
            #print("\n")
                    
    else:   
        last_text= " " + ntext
        

    #print(last_text)
    
    
    jsonData = {}
    jsonData["audio_filepath"] = audioFileName
    #jsonData["duration"] = dduration
    jsonData["text"] = last_text
    json_data = json.dumps(jsonData, ensure_ascii=False)
    mainJsonData.append(json_data)

    #print(jsonData)
    
    #print("\n\n\n")
    

In [ ]:
newJsonFile = "training_files_space.json"
the_file = codecs.open(newJsonFile, "a")#.write(u"\u1234")

cnt = 0

for i in range(0,len(mainJsonData)):
    final = mainJsonData[i]
    the_file.write(str(final))
    the_file.write("\n")
    cnt+=1

the_file.close()

print('Number of total lines of the last json file: ', cnt)

print("THE END")


